<a href="https://colab.research.google.com/github/maachesalah/Malik/blob/main/Self_Adaptive_Learning_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# ============================================
# COMPLETE SELF-ADAPTIVE APPLICATION
# MAPE-K based context-aware application
# Ready to run in Google Colab
# ============================================

# CELL 1: Setup and Imports
!pip install ipywidgets matplotlib numpy -q

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from datetime import datetime
import threading

print("✅ Setup complete - Libraries imported")

# ============================================
# CELL 2: UI Components
# ============================================

class UIComponent:
    """Base class for all UI components that can adapt"""

    def __init__(self, name, component_type):
        self.name = name
        self.component_type = component_type
        self.visible = True
        self.position = 0
        self.size = 'medium'
        self.style = {}

    def adapt(self, context):
        pass

    def render(self):
        pass


class Button(UIComponent):
    def __init__(self, name, label, action):
        super().__init__(name, 'button')
        self.label = label
        self.action = action
        self.button_style = ''  # default
        self.widget = widgets.Button(
            description=label,
            button_style=self.button_style,
            layout=widgets.Layout(width='auto')
        )
        self.widget.on_click(action)

    def adapt(self, context):
        if context.get('user_role') == 'teacher':
            self.button_style = 'success'
            self.widget.button_style = 'success'
        elif context.get('user_role') == 'student':
            self.button_style = 'primary'
            self.widget.button_style = 'primary'
        elif context.get('user_role') == 'admin':
            self.button_style = 'danger'
            self.widget.button_style = 'danger'
        else:
            self.button_style = ''
            self.widget.button_style = ''

        if context.get('page') == 'admin' and 'admin' not in self.name.lower():
            self.visible = False
        else:
            self.visible = True

    def render(self):
        return self.widget if self.visible else None


class Chart(UIComponent):
    def __init__(self, name, data_source):
        super().__init__(name, 'chart')
        self.data_source = data_source
        self.chart_type = 'line'
        self.refresh_rate = 5
        self.widget = widgets.Output()

    def adapt(self, context):
        if context.get('data_volume') == 'large':
            self.chart_type = 'line'
        elif context.get('comparison_needed', False):
            self.chart_type = 'bar'

        if context.get('device') == 'mobile':
            self.refresh_rate = 10
        else:
            self.refresh_rate = 2

    def render(self):
        with self.widget:
            clear_output(wait=True)
            x = np.linspace(0, 10, 50)
            y = np.sin(x) + np.random.normal(0, 0.1, 50)

            plt.figure(figsize=(8, 3))
            if self.chart_type == 'line':
                plt.plot(x, y, 'b-')
            elif self.chart_type == 'bar':
                plt.bar(x[::5], y[::5], color='green', alpha=0.7)

            plt.title(f"{self.name} - {self.chart_type}")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        return self.widget


class TextPanel(UIComponent):
    def __init__(self, name, content):
        super().__init__(name, 'text')
        self.content = content
        self.font_size = 12
        self.language = 'en'
        self.widget = widgets.HTML()

    def adapt(self, context):
        display_text = self.content
        if context.get('user_name'):
            display_text = f"Welcome back, {context['user_name']}!<br>" + display_text

        if context.get('language'):
            self.language = context['language']
            if self.language == 'es':
                display_text = display_text.replace("Welcome", "Bienvenido")

        if context.get('accessibility') == 'high_contrast':
            self.font_size = 16
        elif context.get('device') == 'mobile':
            self.font_size = 14
        else:
            self.font_size = 12

        self.widget.value = f"<div style='font-size: {self.font_size}px; color: black;'>{display_text}</div>"

    def render(self):
        return self.widget


class Menu(UIComponent):
    def __init__(self, name, items):
        super().__init__(name, 'menu')
        self.items = items
        self.visible_items = items.copy()
        self.widget = widgets.Dropdown(
            options=[('Select...', None)] + [(item, item) for item in items],
            description='Menu:',
            disabled=False,
            layout=widgets.Layout(width='300px')
        )

    def adapt(self, context):
        self.visible_items = []
        for item in self.items:
            if context.get('user_role') == 'admin':
                self.visible_items.append(item)
            elif context.get('user_role') == 'teacher' and item not in ['Settings', 'Admin Panel']:
                self.visible_items.append(item)
            elif context.get('user_role') == 'student' and item in ['Dashboard', 'Courses', 'Grades', 'Help']:
                self.visible_items.append(item)

        if self.visible_items:
            self.widget.options = [('Select...', None)] + [(item, item) for item in self.visible_items]
        else:
            self.widget.options = [('No items', None)]

    def render(self):
        return self.widget


# ============================================
# CELL 3: Context Manager
# ============================================

class ContextManager:
    def __init__(self):
        self.context_sources = []
        self.current_context = {
            'user_name': 'Student',
            'user_role': 'student',
            'user_location': 'classroom',
            'user_activity': 'learning',
            'device': 'desktop',
            'screen_size': 'large',
            'battery_level': 100,
            'network': 'wifi',
            'time_of_day': datetime.now().hour,
            'day_of_week': datetime.now().weekday(),
            'location': 'campus',
            'noise_level': 'low',
            'app_state': 'active',
            'page': 'dashboard',
            'data_volume': 'medium',
            'performance': 1.0,
            'accessibility': 'none',
            'language': 'en',
            'comparison_needed': False
        }
        self.context_history = []

    def update(self):
        self.current_context['time_of_day'] = datetime.now().hour
        if random.random() < 0.1:
            self.simulate_context_change()
        self.context_history.append(self.current_context.copy())
        return self.current_context

    def simulate_context_change(self):
        change_type = random.choice(['user', 'device', 'environment', 'app'])

        if change_type == 'user':
            self.current_context['user_activity'] = random.choice(['learning', 'teaching', 'grading', 'planning'])
            if random.random() < 0.3:
                self.current_context['user_role'] = random.choice(['student', 'teacher', 'admin'])

        elif change_type == 'device':
            self.current_context['device'] = random.choice(['desktop', 'mobile', 'tablet'])
            self.current_context['network'] = random.choice(['wifi', '4g', 'offline'])
            self.current_context['battery_level'] = random.randint(10, 100)

        elif change_type == 'environment':
            self.current_context['location'] = random.choice(['campus', 'home', 'library', 'cafe'])
            self.current_context['noise_level'] = random.choice(['low', 'medium', 'high'])

        elif change_type == 'app':
            self.current_context['page'] = random.choice(['dashboard', 'courses', 'grades', 'settings'])
            self.current_context['data_volume'] = random.choice(['low', 'medium', 'large'])

    def get_context(self):
        return self.current_context.copy()

    def display_context(self):
        print("\n📊 CURRENT CONTEXT:")
        for key, value in self.current_context.items():
            print(f"   {key}: {value}")


# ============================================
# CELL 4: Adaptation Engine (MAPE-K)
# ============================================

class AdaptationEngine:
    def __init__(self, context_manager):
        self.context_manager = context_manager
        self.components = []
        self.adaptation_rules = self.load_rules()
        self.adaptation_history = []

    def load_rules(self):
        return [
            {
                'condition': lambda ctx: ctx.get('device') == 'mobile',
                'adaptation': 'simplify_ui',
                'priority': 'high'
            },
            {
                'condition': lambda ctx: ctx.get('battery_level', 100) < 20,
                'adaptation': 'conserve_power',
                'priority': 'high'
            },
            {
                'condition': lambda ctx: ctx.get('user_role') == 'teacher',
                'adaptation': 'show_teacher_tools',
                'priority': 'medium'
            },
            {
                'condition': lambda ctx: ctx.get('network') == 'offline',
                'adaptation': 'offline_mode',
                'priority': 'high'
            },
            {
                'condition': lambda ctx: ctx.get('noise_level') == 'high',
                'adaptation': 'increase_font',
                'priority': 'low'
            },
            {
                'condition': lambda ctx: ctx.get('data_volume') == 'large',
                'adaptation': 'summarize_data',
                'priority': 'medium'
            },
            {
                'condition': lambda ctx: ctx.get('time_of_day') > 20 or ctx.get('time_of_day') < 6,
                'adaptation': 'dark_mode',
                'priority': 'low'
            },
            {
                'condition': lambda ctx: ctx.get('user_role') == 'admin',
                'adaptation': 'full_access',
                'priority': 'medium'
            }
        ]

    def register_component(self, component):
        self.components.append(component)
        print(f"  ✅ Registered: {component.name}")

    def monitor(self):
        return self.context_manager.get_context()

    def analyze(self, context):
        applicable_rules = []
        for rule in self.adaptation_rules:
            try:
                if rule['condition'](context):
                    applicable_rules.append(rule)
            except:
                pass
        return applicable_rules

    def plan(self, applicable_rules, context):
        if not applicable_rules:
            return None

        priority_order = {'high': 3, 'medium': 2, 'low': 1}
        applicable_rules.sort(key=lambda r: priority_order.get(r['priority'], 0), reverse=True)

        selected_adaptations = []
        for rule in applicable_rules[:3]:
            selected_adaptations.append(rule['adaptation'])

        return {
            'adaptations': selected_adaptations,
            'timestamp': time.time(),
            'context': context.copy()
        }

    def execute(self, plan):
        if not plan:
            return

        context = plan['context']
        for component in self.components:
            try:
                component.adapt(context)
            except:
                pass

        self.adaptation_history.append(plan)
        print(f"\n🔄 ADAPTATION: {plan['adaptations']}")

    def run_mapek_cycle(self):
        context = self.monitor()
        applicable_rules = self.analyze(context)
        plan = self.plan(applicable_rules, context)
        if plan:
            self.execute(plan)
        return plan


# ============================================
# CELL 5: Self-Adaptive Application
# ============================================

class SelfAdaptiveApp:
    def __init__(self):
        self.context_manager = ContextManager()
        self.adaptation_engine = AdaptationEngine(self.context_manager)
        self.components = []
        self.create_components()
        for component in self.components:
            self.adaptation_engine.register_component(component)
        self.main_container = widgets.VBox([])
        self.status_bar = widgets.HTML("🟢 System Ready")
        self.auto_adapt = True
        print("✅ Self-Adaptive App initialized!")

    def create_components(self):
        # Menu
        self.menu = Menu("Main Menu", ['Dashboard', 'Courses', 'Grades', 'Analytics', 'Settings', 'Help', 'Admin Panel'])
        self.components.append(self.menu)

        # Welcome panel
        self.welcome = TextPanel("Welcome", "Welcome to the Self-Adaptive Learning Dashboard!<br>This application adapts itself based on your context.")
        self.components.append(self.welcome)

        # Buttons
        self.btn_dashboard = Button("Dashboard", "📊 View Dashboard", self.on_dashboard)
        self.components.append(self.btn_dashboard)

        self.btn_courses = Button("Courses", "📚 My Courses", self.on_courses)
        self.components.append(self.btn_courses)

        self.btn_grades = Button("Grades", "📝 Check Grades", self.on_grades)
        self.components.append(self.btn_grades)

        self.btn_analytics = Button("Analytics", "📈 Learning Analytics", self.on_analytics)
        self.components.append(self.btn_analytics)

        # Chart
        self.chart = Chart("Learning Progress", "student_data")
        self.components.append(self.chart)

        # Info panel
        self.info_panel = TextPanel("Info", "Current context information will appear here.")
        self.components.append(self.info_panel)

        # Control buttons
        self.btn_adapt = Button("Adapt", "🔄 Force Adaptation", self.force_adapt)
        self.components.append(self.btn_adapt)

        self.btn_context = Button("Context", "📊 Show Context", self.show_context)
        self.components.append(self.btn_context)

        self.btn_auto = Button("Auto", "⚡ Toggle Auto-Adapt", self.toggle_auto)
        self.components.append(self.btn_auto)

    def on_dashboard(self, btn):
        self.context_manager.current_context['page'] = 'dashboard'
        self.update_display("📊 Dashboard view activated")

    def on_courses(self, btn):
        self.context_manager.current_context['page'] = 'courses'
        self.update_display("📚 Courses view activated")

    def on_grades(self, btn):
        self.context_manager.current_context['page'] = 'grades'
        self.update_display("📝 Grades view activated")

    def on_analytics(self, btn):
        self.context_manager.current_context['page'] = 'analytics'
        self.update_display("📈 Analytics view activated")

    def force_adapt(self, btn):
        self.adaptation_engine.run_mapek_cycle()
        self.render()
        self.update_display("🔄 Manual adaptation triggered")

    def show_context(self, btn):
        self.context_manager.display_context()

    def toggle_auto(self, btn):
        self.auto_adapt = not self.auto_adapt
        status = "ON" if self.auto_adapt else "OFF"
        self.update_display(f"⚡ Auto-adapt: {status}")

    def update_display(self, message):
        self.status_bar.value = f"🟢 {message} | {datetime.now().strftime('%H:%M:%S')}"

    def auto_adapt_loop(self):
        while self.auto_adapt:
            time.sleep(5)
            if self.auto_adapt:
                self.context_manager.update()
                self.adaptation_engine.run_mapek_cycle()
                self.render()

    def render(self):
        visible_components = [self.status_bar]

        menu_widget = self.menu.render()
        if menu_widget:
            visible_components.append(menu_widget)

        welcome_widget = self.welcome.render()
        if welcome_widget:
            visible_components.append(welcome_widget)

        button_row = widgets.HBox([
            self.btn_dashboard.render(),
            self.btn_courses.render(),
            self.btn_grades.render(),
            self.btn_analytics.render()
        ])
        visible_components.append(button_row)

        chart_widget = self.chart.render()
        if chart_widget:
            visible_components.append(chart_widget)

        info_widget = self.info_panel.render()
        if info_widget:
            visible_components.append(info_widget)

        control_row = widgets.HBox([
            self.btn_adapt.render(),
            self.btn_context.render(),
            self.btn_auto.render()
        ])
        visible_components.append(control_row)

        self.main_container.children = visible_components
        clear_output(wait=True)
        display(self.main_container)

    def run(self):
        print("🚀 Starting Self-Adaptive Application...")
        print("The app adapts every 5 seconds based on context")
        print("-" * 50)
        self.render()
        thread = threading.Thread(target=self.auto_adapt_loop, daemon=True)
        thread.start()


# ============================================
# CELL 6: Manual Control Panel
# ============================================

class ControlPanel:
    def __init__(self, app):
        self.app = app

    def create(self):
        print("🎮 MANUAL CONTEXT CONTROLS")

        def set_role(role):
            self.app.context_manager.current_context['user_role'] = role
            self.app.adaptation_engine.run_mapek_cycle()
            self.app.render()
            self.app.update_display(f"👤 Role: {role}")

        def set_device(device):
            self.app.context_manager.current_context['device'] = device
            self.app.adaptation_engine.run_mapek_cycle()
            self.app.render()
            self.app.update_display(f"📱 Device: {device}")

        def set_network(network):
            self.app.context_manager.current_context['network'] = network
            self.app.adaptation_engine.run_mapek_cycle()
            self.app.render()
            self.app.update_display(f"🌐 Network: {network}")

        def set_location(location):
            self.app.context_manager.current_context['location'] = location
            self.app.adaptation_engine.run_mapek_cycle()
            self.app.render()
            self.app.update_display(f"📍 Location: {location}")

        def set_accessibility(mode):
            self.app.context_manager.current_context['accessibility'] = mode
            self.app.adaptation_engine.run_mapek_cycle()
            self.app.render()
            self.app.update_display(f"♿ Accessibility: {mode}")

        role_buttons = widgets.HBox([
            widgets.Button(description="👤 Student", button_style='primary',
                          on_click=lambda x: set_role('student')),
            widgets.Button(description="👨‍🏫 Teacher", button_style='success',
                          on_click=lambda x: set_role('teacher')),
            widgets.Button(description="👑 Admin", button_style='danger',
                          on_click=lambda x: set_role('admin'))
        ])

        device_buttons = widgets.HBox([
            widgets.Button(description="💻 Desktop", on_click=lambda x: set_device('desktop')),
            widgets.Button(description="📱 Mobile", on_click=lambda x: set_device('mobile')),
            widgets.Button(description="📟 Tablet", on_click=lambda x: set_device('tablet'))
        ])

        network_buttons = widgets.HBox([
            widgets.Button(description="📶 WiFi", on_click=lambda x: set_network('wifi')),
            widgets.Button(description="📱 4G", on_click=lambda x: set_network('4g')),
            widgets.Button(description="✈️ Offline", on_click=lambda x: set_network('offline'))
        ])

        location_buttons = widgets.HBox([
            widgets.Button(description="🏫 Campus", on_click=lambda x: set_location('campus')),
            widgets.Button(description="🏠 Home", on_click=lambda x: set_location('home')),
            widgets.Button(description="📚 Library", on_click=lambda x: set_location('library'))
        ])

        accessibility_buttons = widgets.HBox([
            widgets.Button(description="👁️ Normal", on_click=lambda x: set_accessibility('none')),
            widgets.Button(description="🔍 High Contrast", on_click=lambda x: set_accessibility('high_contrast')),
            widgets.Button(description="🔊 Screen Reader", on_click=lambda x: set_accessibility('screen_reader'))
        ])

        display(widgets.VBox([
            widgets.HTML("<h3>👤 User Role</h3>"),
            role_buttons,
            widgets.HTML("<h3>📱 Device Type</h3>"),
            device_buttons,
            widgets.HTML("<h3>🌐 Network</h3>"),
            network_buttons,
            widgets.HTML("<h3>📍 Location</h3>"),
            location_buttons,
            widgets.HTML("<h3>♿ Accessibility</h3>"),
            accessibility_buttons
        ]))


# ============================================
# CELL 7: Adaptation History Viewer
# ============================================

def show_adaptation_history(app):
    if not app.adaptation_engine.adaptation_history:
        print("No adaptations yet")
        return

    print("📜 ADAPTATION HISTORY (last 10)")
    print("=" * 60)

    for i, adaptation in enumerate(app.adaptation_engine.adaptation_history[-10:]):
        print(f"\n{i+1}. Time: {datetime.fromtimestamp(adaptation['timestamp']).strftime('%H:%M:%S')}")
        print(f"   Adaptations: {', '.join(adaptation['adaptations'])}")
        print(f"   Context: {adaptation['context'].get('user_role', 'unknown')} on {adaptation['context'].get('device', 'unknown')}")


# ============================================
# CELL 8: Performance Tracker
# ============================================

class PerformanceTracker:
    def __init__(self, app):
        self.app = app
        self.metrics = {
            'adaptation_count': [],
            'response_times': [],
            'context_changes': []
        }

    def measure_response_time(self):
        start = time.time()
        self.app.adaptation_engine.run_mapek_cycle()
        end = time.time()
        return end - start

    def collect_metrics(self):
        self.metrics['adaptation_count'].append(len(self.app.adaptation_engine.adaptation_history))
        self.metrics['response_times'].append(self.measure_response_time())
        self.metrics['context_changes'].append(len(self.app.context_manager.context_history))

    def plot_metrics(self):
        if not self.metrics['response_times']:
            print("No metrics yet")
            return

        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(self.metrics['response_times'], 'g-', linewidth=2)
        plt.title('Adaptation Response Time')
        plt.xlabel('Measurement')
        plt.ylabel('Seconds')
        plt.grid(True, alpha=0.3)

        plt.subplot(1, 2, 2)
        plt.bar(['Adaptations'], [self.metrics['adaptation_count'][-1]], color='blue', alpha=0.7)
        plt.title('Total Adaptations')
        plt.ylabel('Count')
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()


# ============================================
# CELL 9: RUN THE APPLICATION
# ============================================

print("=" * 60)
print("🚀 SELF-ADAPTIVE APPLICATION WITH MAPE-K")
print("=" * 60)

# Create and run app
app = SelfAdaptiveApp()
app.run()

print("\n📝 INSTRUCTIONS:")
print("1. Watch the app auto-adapt every 5 seconds")
print("2. Use the control panel below to manually change context")
print("3. Click 'Show Context' to see current state")
print("4. Click 'Force Adaptation' for manual adaptation")
print("5. Toggle 'Auto' to enable/disable automatic adaptation")
print("=" * 60)


# ============================================
# CELL 10: Launch Control Panel
# ============================================

# Create and display control panel
panel = ControlPanel(app)
panel.create()


# ============================================
# CELL 11: View Adaptation History
# ============================================

show_adaptation_history(app)


# ============================================
# CELL 12: Track Performance
# ============================================

# Create tracker
tracker = PerformanceTracker(app)

# Collect some metrics
for _ in range(3):
    tracker.collect_metrics()
    time.sleep(1)

# Show results
tracker.plot_metrics()


# ============================================
# CELL 13: Export Functions for Research
# ============================================

def export_adaptation_log(app, filename="adaptation_log.txt"):
    """Export adaptation history to file"""
    with open(filename, 'w') as f:
        f.write("ADAPTATION LOG\n")
        f.write("=" * 50 + "\n")
        for i, adaptation in enumerate(app.adaptation_engine.adaptation_history):
            f.write(f"\n{i+1}. Time: {datetime.fromtimestamp(adaptation['timestamp'])}\n")
            f.write(f"   Adaptations: {', '.join(adaptation['adaptations'])}\n")
            f.write(f"   Context: {adaptation['context']}\n")
    print(f"✅ Log exported to {filename}")

def calculate_adaptation_frequency(app):
    """Calculate adaptations per minute"""
    if len(app.adaptation_engine.adaptation_history) < 2:
        return 0

    first = app.adaptation_engine.adaptation_history[0]['timestamp']
    last = app.adaptation_engine.adaptation_history[-1]['timestamp']
    duration_minutes = (last - first) / 60

    if duration_minutes > 0:
        return len(app.adaptation_engine.adaptation_history) / duration_minutes
    return 0

print(f"\n📊 Adaptation Frequency: {calculate_adaptation_frequency(app):.2f} adaptations/minute")